# Etap 1.1.2
## Wczytanie surowych danych dla każdego kwartału, tłumaczenie nazw kolumn
## zapis do pliku ""Data/Concatenated Features/by_quarters_features.csv"


In [1]:
# run only once
#!pip install deep-translator

In [2]:
import pandas as pd
import numpy as np
import os
from glob import glob
from deep_translator import GoogleTranslator


In [3]:
# Ścieżka do folderu z plikami Excel
folder_path = "Data/Turkish Public Companies Balance Sheets from KAP"

# Lista wszystkich plików .xlsx w folderze
excel_files = glob(os.path.join(folder_path, "*.xlsx"))

# Lista do zebrania przetworzonych DataFrame'ów
df_list = []

for file in excel_files:
    # Wczytaj plik
    df = pd.read_excel(file)
    
    # Filtrowanie tylko na interesujące kolumny czasowe
    periods = ['Yıllık', '9 Aylık', '6 Aylık', '3 Aylık']
    
    # Pivot dla każdej kolumny okresu
    df_wide_parts = []
    for period in periods:
        df_pivot = df.pivot_table(index=[], columns='Desc', values=period, aggfunc='first').reset_index()
        # Zmiana nazw kolumn: dodaj prefiks okresu
        df_pivot.columns = [f"{period}_{col}" if col != '' else col for col in df_pivot.columns]
        df_wide_parts.append(df_pivot)
    
    # Połączenie szerokich danych dla różnych okresów
    df_wide = pd.concat(df_wide_parts, axis=1)
    
    # Dodaj kolumnę z nazwą pliku (bez ścieżki)
    df_wide['source_file'] = os.path.basename(file)
    
    # Dodaj do listy
    df_list.append(df_wide)

# Połącz wszystkie w jeden DataFrame
final_df = pd.concat(df_list, ignore_index=True)


final_df['Nazwa'] = final_df['source_file'].str.extract(r'^([^_]+)')[0]
final_df['Rok'] = final_df['source_file'].str.extract(r'_([0-9]{4})')[0]





C:\Users\User\AppData\Local\Temp\ipykernel_14840\3683679992.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df['Nazwa'] = final_df['source_file'].str.extract(r'^([^_]+)')[0]
C:\Users\User\AppData\Local\Temp\ipykernel_14840\3683679992.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df['Rok'] = final_df['source_file'].str.extract(r'_([0-9]{4})')[0]


In [6]:
final_df.head()

,Yıllık_index,Yıllık_ (Ara Toplam),Yıllık_ Uzun vadeli karşılıklar,Yıllık_ Amortisman & İtfa Payları,Yıllık_ Ana Ortaklığa Ait Özkaynaklar,Yıllık_ Azınlık Payları,Yıllık_ Borç Karşılıkları,Yıllık_ Canlı Varlıklar,Yıllık_ Devlet Teşvik ve Yardımları,Yıllık_ Değer Artış Fonları,...,3 Aylık_Ticari Faaliyetlerden Diğer Kar (Zarar),3 Aylık_Uzun Vadeli Yükümlülükler,3 Aylık_Vergi Öncesi Diğer Gelir ve Giderler,3 Aylık_Yab. Para Çev. Fark. Etk. Önc.Nak.Ve Nak. Benz. Net Artış/Azalış,3 Aylık_Yurtdışı Satışlar,3 Aylık_Yurtiçi Satışlar,3 Aylık_Özkaynak Yöntemiyle Değerlenen Yatırımların Kar/Zararlarındaki Paylar,3 Aylık_Özkaynaklar,Nazwa,Rok
0,Yıllık,4.524640e+08,678404.0,1242248.0,1.387926e+08,0.0,2852756.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1CAP,2020
1,Yıllık,8.515088e+08,0.0,3263754.0,2.966258e+08,0.0,6670834.0,0.0,0.0,0.0,...,0.0,2968732.0,21092.0,6074430.0,0.0,0.0,0.0,1.600191e+08,A1CAP,2021
2,Yıllık,4.019261e+09,4748296.0,12651941.0,1.487849e+09,0.0,70752251.0,0.0,0.0,0.0,...,0.0,11317451.0,26841.0,-29735610.0,0.0,0.0,0.0,3.671349e+08,A1CAP,2022
3,Yıllık,6.297396e+09,7715324.0,21452328.0,3.696944e+09,0.0,42331251.0,0.0,0.0,0.0,...,0.0,40441965.0,127664.0,247043885.0,0.0,0.0,0.0,9.180652e+08,A1CAP,2023
4,Yıllık,8.297508e+09,9773361.0,31826374.0,4.065838e+09,0.0,28062814.0,0.0,0.0,0.0,...,0.0,56364509.0,1243146.0,627250177.0,0.0,0.0,0.0,3.051387e+09,A1CAP,2024


In [7]:
from deep_translator import GoogleTranslator, exceptions

translated_columns = []
translator = GoogleTranslator(source='tr', target='pl')

for col in final_df.columns:
    # Zamień _ na spacje, usuń podwójne spacje, usuń zbędne spacje na końcach
    text_to_translate = col.replace('_', ' ')
    text_to_translate = ' '.join(text_to_translate.split())  # Usuwa nadmiarowe spacje

    try:
        translated = translator.translate(text_to_translate)
        # Po tłumaczeniu zamień spacje z powrotem na podkreślenia
        translated = translated.replace(' ', '_')
    except exceptions.TranslationNotFound:
        # Jeśli tłumaczenie nie działa, zachowaj oryginał z zamianą spacji
        print(f"⚠️ Nie znaleziono tłumaczenia dla: {text_to_translate}, pozostawiam oryginał.")
        translated = col  # Zostaw oryginał

    translated_columns.append(translated)

# Przypisz nowe kolumny
final_df.columns = translated_columns

# Wyświetl wynik
print("Przetłumaczone kolumny:")
len( translated_columns)
final_df.head()

Przetłumaczone kolumny:


,Indeks_roczny,Roczna_(suma_pośrednia),Coroczne_przepisy_długoterminowe,Coroczne_amortyzacja_i_straż_pożarna,Kapitał_rocznego_partnerstwa_głównego,Roczne_akcje_mniejszościowe,Postanowienia_roczne,Coroczne_aktywa_mieszkalne,Coroczne_zachęty_państwowe_i_pomoc,Roczny_wzrost_wartości_wzrostu,...,Inne_zyski_z_3_-miesięcznej_działalności_komercyjnej_(szkody),3_-miesięczne_zobowiązania_długoterminowe,Inne_dochody_i_wydatki_przed_3_-miesięcznym_podatkiem,3_miesiące_yab._Money_Trans._Różnica._Efekt._Poprzedni.nak.ve_nak._Tak_jak._Wzrost/spadek_netto,3_miesiące_za_granicą,3_miesiące_sprzedaży_krajowej,Akcje_zysków/strat_inwestycji_o_wartości_3_-miesięcznej_kapitału_własnego,3_miesiące_siebie,Nazwa,Rok
0,Yıllık,4.524640e+08,678404.0,1242248.0,1.387926e+08,0.0,2852756.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1CAP,2020
1,Yıllık,8.515088e+08,0.0,3263754.0,2.966258e+08,0.0,6670834.0,0.0,0.0,0.0,...,0.0,2968732.0,21092.0,6074430.0,0.0,0.0,0.0,1.600191e+08,A1CAP,2021
2,Yıllık,4.019261e+09,4748296.0,12651941.0,1.487849e+09,0.0,70752251.0,0.0,0.0,0.0,...,0.0,11317451.0,26841.0,-29735610.0,0.0,0.0,0.0,3.671349e+08,A1CAP,2022
3,Yıllık,6.297396e+09,7715324.0,21452328.0,3.696944e+09,0.0,42331251.0,0.0,0.0,0.0,...,0.0,40441965.0,127664.0,247043885.0,0.0,0.0,0.0,9.180652e+08,A1CAP,2023
4,Yıllık,8.297508e+09,9773361.0,31826374.0,4.065838e+09,0.0,28062814.0,0.0,0.0,0.0,...,0.0,56364509.0,1243146.0,627250177.0,0.0,0.0,0.0,3.051387e+09,A1CAP,2024


In [8]:
# Póki co nie wykorzystane, tylko ocena poziomu braku danych

q = 0.4

# Oblicz odsetek NaN w każdej kolumnie
nan_ratio = final_df.isna().mean()


# Wybierz kolumny do odrzucenia (mające więcej niż q NaN)
columns_to_drop = nan_ratio[nan_ratio > q].index


print("\n❌ Kolumny, które zostaną odrzucone (> {:.0%} NaN):".format(q))
print(list(columns_to_drop))
len(columns_to_drop)


❌ Kolumny, które zostaną odrzucone (> 40% NaN):
['Coroczne_odroczone_przychody_(obietnica_klienta._Dogan_Load._Zewnętrzna._Kal.)', 'Coroczne_odroczone_przychody_(klient_Söz.doğan_Yür.', 'Aktywa_o_prawach_rocznych', 'Coroczna_obietnica_klientów._Obciążenie_Dogan.', 'Roczne_słowo_klienta.', 'Aktywa_wynikające_z_rocznych_umów_o_klientach', 'Zysk/strata_aktywności_przed_rocznym_kosztem_finansowania', '9_miesięcy_amortyzacja_i_straż_pożarna', 'Inne_przychody_i_wydatki_9_miesięcy', '9_miesięcy_innych_dochodów/_kosztów', 'Gotówka_z_innych_9_-miesięcznych_działań_inwestycyjnych', 'Gotówka_z_innych_9_-miesięcznych_działań_biznesowych', '9_-miesięczne_odroczone_przychody_(obietnica_klienta._Doğan_Yür._Exterior.Kal.)', '9_-miesięczne_odroczone_przychody_(Word_klienta.', 'Zmiana_w_9_miesiącach_rezerw', '9_miesięcy_właściwy_zespół', '9_-Mentowa_obietnica_klienta._Obciążenie_Dogan.', '9_-miesięczne_słowo_klienta.', 'Aktywa_wynikające_z_9_-miesięcznych_umów_klienta', '9_-miesięczne_stałe_inwestycje_

88

In [9]:
final_df.to_csv("Data/Concatenated Features/by_quaters_features.csv", index=False, encoding='utf-8-sig')